<h1 "style=color:blue"> Chapter 5 </h1>

<h2 "style = color:red"> Time Value of Money </h2>

In [ ]:
cashflows=[100,200,300,400,500]
r = 0.1

In [ ]:
def find_pv(r,cashflows):
    pv = 0
    for i in range(len(cashflows)):
        pv = pv + cashflows[i]/(1+r)**(i+1)
    return (pv)

In [ ]:
find_pv(r,cashflows)

In [ ]:
def find_pva(r,k,A):
    pva = (A/r)*(1-(1/(1+r)**k))
    return pva

In [ ]:
find_pva(0.1,5,100)

In [ ]:
def find_fv(r,cashflows):
    k=len(cashflows)
    fv = 0
    for i in range(k):
        fv = fv + cashflows[i]*(1+r)**(k-i-1)
    return fv
    

In [ ]:
find_fv(r,cashflows)

In [ ]:
def find_fva(r,k,A):
    fva = (A/r)*((1+r)**k-1)
    return fva

In [ ]:
find_fva(0.1,5,100)

<h2 "style = color:red"> Risk and Return </h2>

In [ ]:
import pandas as pd

In [ ]:
prices = pd.Series([100,101,98,83,97])
discrete_returns = prices.pct_change().dropna()
discrete_returns

In [ ]:
import numpy as np
log_returns = np.log(1+discrete_returns)
log_returns

In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.famafrench as ff

In [ ]:
ff_data = ff.get_available_datasets()

In [ ]:
ff_data[:5]

In [ ]:
ff_factors = ff.FamaFrenchReader("F-F_Research_Data_Factors").read()

In [ ]:
ff_3f = ff_factors[0]
ff_3f.head(2)

In [ ]:
import yfinance as yf

In [ ]:
ibm = yf.download("IBM")

In [ ]:
ibm.head(2)

In [ ]:
ibm_monthly = ibm.resample('MS').last()
ibm_monthly.head(2)

In [ ]:
ibm_monthly['returns']=ibm_monthly['Adj Close'].pct_change()*100

In [ ]:
ff_3f.index = ff_3f.index.to_timestamp(how='start')
ff_3f.head(2)

In [ ]:
ibm_ff = ibm_monthly.join(ff_3f)

In [ ]:
ibm_ff = ibm_ff.loc["2017":"2021",['returns','Mkt-RF','SMB','HML','RF']].dropna()
ibm_ff.head(2)

In [ ]:
ff_3f.index.dtype

In [ ]:
pd.Period(year=2022,month=3,freq="M")

In [ ]:
pd.Period(year=2022,month=3,freq="M").to_timestamp(how='start')

In [ ]:
import statsmodels.api as sm

In [ ]:
ibm_ff['excess_returns'] = ibm_ff['returns']-ibm_ff['RF']

In [ ]:
X = ibm_ff[['Mkt-RF','SMB','HML']]
y = ibm_ff['excess_returns']

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(y,X).fit()

In [ ]:
model.summary()

In [ ]:
model.params

<h2 "style=color:red"> Bond Valuation <?h2>

In [ ]:
C = 10
n = 5
y = 0.08
FV = 100

PV = C/y*(1-1/(1+y)**n)+FV/(1+y)**n
PV

In [ ]:
def find_yield(FV,n,C,MV):
    y = 0.1 #initial value
    PV = C/y*(1-1/(1+y)**n)+FV/(1+y)**n
    while abs(MV-PV)>0.01:
        if MV>PV:
            y = y - 0.0001
            PV = C/y*(1-1/(1+y)**n)+FV/(1+y)**n
        else:
            y = y + 0.0001
            PV = C/y*(1-1/(1+y)^n)+FV/(1+y)^n
    return y

In [ ]:
find_yield(FV=100,n=5,C=10,MV=107.9854)

In [ ]:
def bond_value(FV, r, C):
    n = len(r)
    PV = 0
    for i in range(1,n+1):
        PV = PV + C/(1+r[i-1])**i
  
    PV = PV + FV/(1+r[n-1])**n
    return PV


In [ ]:
r = [0.04, 0.05, 0.06]
FV = 100
C = 10
bond_value(FV,r,C)

In [ ]:
find_yield(FV=100,n=3,C=10,MV=111.0438)

<h2 "style=color:red"> Stock Valuation </h2>

In [ ]:
def value_share(D0,g_current,g_terminal,r,nYears):
    div=[0]*(1+nYears)
    div[0]=D0
    value=0
    for i in range(1,nYears+1):
        div[i]=div[i-1]*(1+g_current)
        value = value + div[i]/(1+r)**i
    terminal_value = div[-1]*(1+g_terminal)/(r-g_terminal)
    value = value + terminal_value/(1+r)**nYears
    return value    

In [ ]:
value_share(2.20,0.0853,0.02,0.0355,5)

In [ ]:
def value_share_g(networth,ROE,r,g,n):
    payout = 1 - g/ROE
    net_income = networth * ROE
    dividend_total = net_income * payout
    dividend_per_share = dividend_total/n
    price = dividend_per_share/(r-g)
    return price

In [ ]:
value_share_g(100,0.2,0.15,0.05,3)

In [ ]:
value_share_g(100,0.2,0.15,0.1,3)

<h2 "style=color:red"> Option Payoff Diagrams </h2>

In [ ]:
def call_option(S,K):
    return np.maximum(S-K,0)
def put_option(S,K):
    return np.maximum(K-S,0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def put_payoff_diagram(S,K):
    put_values = put_option(S,K)
    sns.lineplot(x=S,y=put_values)
    plt.xlabel("Stock Prices")
    plt.ylabel("Option Payoffs")
    plt.title("Option Payoff to Put Option")    

In [ ]:
ST = np.arange(80,120)
put_payoff_diagram(S=ST,K=100)

<h2 style="color:red"> Black Scholes </h2>

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def wiener_sim(del_t=0.001,n=100):
    z = np.random.randn(n)*np.sqrt(del_t)
    return np.cumsum(z)

In [ ]:
plt.plot(wiener_sim())
plt.xlabel("Time")
plt.title("Stochastic Processing following Wiener Process");

In [ ]:
def gen_wiener_sim(a=10,b=3,del_t=0.001,n=100):
    w = np.random.normal(a*del_t,b*np.sqrt(del_t),n)
    return np.cumsum(w)

In [ ]:
z_init = 100
plt.plot(z_init + gen_wiener_sim(a=10,b=10));

<h2 "style=color:red"> Simulation to value options </h2>

In [ ]:
def simulate_stock_prices(S,r,t,sigma,n=1000000):
    z = np.random.randn(n)
    ST = S*np.exp((r-sigma*sigma/2)*t + sigma*np.sqrt(t)*z)
    return ST    

In [ ]:
sns.histplot(simulate_stock_prices(S=100,r=0.03,t=0.25,sigma=0.3));

In [ ]:
def call_sim(S,K,r,t,sigma,n=1000000):
    call_T = np.mean(np.maximum(simulate_stock_prices(S=S,r=r,t=t,sigma=sigma,n=n)-K,0))
    call = np.exp(-r*t)*call_T
    return call

In [ ]:
call_sim(S=100,K=90,r=0.03,t=0.25,sigma=0.3)

In [ ]:
def BS_Call(S,K,r,t,sigma):
    d1 = (np.log(S/K)+(r+sigma*sigma/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    nd1 = scipy.stats.norm.cdf(d1)
    nd2 = scipy.stats.norm.cdf(d2)
    call = S*nd1 - K*np.exp(-r*t)*nd2
    return call

In [ ]:
BS_Call(S=100,K=90,r=0.03,t=0.25,sigma=0.3)

In [ ]:
def implied_vol(S,K,r,t,call):
    sigma = 0.5
    call_est = BS_Call(S,K,r,t,sigma)
    while abs(call-call_est)>0.001:
        call_est = BS_Call(S,K,r,t,sigma)
        if call>call_est:
            sigma = sigma + 0.0001
        else:
            sigma = sigma - 0.0001
    return sigma

In [ ]:
implied_vol(S=100,K=90,r=0.03,t=0.25,call=12.52)

In [ ]:
S = np.arange(80,130)
options = [BS_Call(S=i,K=100,r=0.03,t=0.25,sigma=0.3) for i in S]
callT = np.maximum(S-100,0)
sns.lineplot(x=S,y=options-callT)
plt.title("Time Premiums in Options")
plt.xlabel("Stock Prices")
plt.ylabel("Time Premiums")
plt.vlines(x=100,ymin = 0,ymax=7);